In [2]:
from sage.graphs.connectivity import connected_components
from sage.graphs.connectivity import connected_components_number

#m x n mreža, a št. izbrisanih vozlov, b št.izrbirsanih povezav
def mreza(m,n,a,b):
    mreza = graphs.Grid2dGraph(m,n)
    if a > mreza.order():
        print("Za ukaz je na voljo premalo vozlov.")
    else:
        i = 0
        while i < a:
            mreza.delete_vertex(mreza.random_vertex())
            i = i+1
        i = 0
    if b > mreza.size():
        print("Za ukaz je na voljo premalo povezav.")
    else:
        while i < b:
            mreza.delete_edge(mreza.random_edge())
            i = i+1
    return mreza

#reši CLP problem in vrne k
def najkrajsa_razdalja(G, st_centrov):
    K = st_centrov
    razdalje = G.distance_all_pairs()

    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable(binary=True) #x_uv = 1 če mesto u spada k skladišču v (mestu v s skladiščem)
    y = p.new_variable(binary=True) # y_v = 1 če je v mestu v skladišče

    p.set_objective(p['R']) # največja razdalja je spremenljivka

    for u in G:
        p.add_constraint(sum(x[u, v] for v in G) == 1) #za vsako mesto u bo veljalo, da spada pod neko območje mesta v s skladiščem

    p.add_constraint(sum(y[v] for v in G) == K) #vsota skladišč je enaka K

    for u in G:
        for v in G:
            p.add_constraint(x[u, v] <= y[v]) #ne sme se zgoditi, da mesto u pade v območje mesta v, v mestu v pa sploh ni skladišča

    for u in G:
        for v in G:
            if v in razdalje[u]:
                p.add_constraint(razdalje[u][v] * x[u, v] <= p['R']) # če sta vozlišči v isti povezani komponenti, potem omejimo največjo razdaljo                                                                        do skladišča
            else:
                p.add_constraint(x[u, v] == 0) # sicer mesto u ne more pripadati skladišču v
    max_razdalja = p.solve()
    skladisca = [k for k, v in p.get_values(y).items() if v == 1]
    #print(skladisca)
    return max_razdalja


#G = mreza(5,3,3,2)
#slika=G.show()
#najkrajsa_razdalja(G,5)

#print(connected_components(G)) #večdelni graf zapiše po ločenih delih
#print(connected_components_number(G)) #št. delov

In [2]:
#kako se optimalna vrednost spreminja glede na k
def opt_vrednost_k(m,n,a,b,k):
    G = mreza(m,n,a,b)
    seznam_vrednosti = []
    stevilo_komponent = connected_components_number(G)
    for i in range(stevilo_komponent,k+1): ## težave pri določanju najmanjšega i???
        razdalja = round(najkrajsa_razdalja(G, i)) #round za zaokrževanje števil
        seznam_vrednosti.append((razdalja))
    return seznam_vrednosti

#vrne optimalno vrednost R za več ponovitev
#FIKSNO: velikost mreže, število izbrisanih povezav in vozlišč
#SPREMINJAVA: k
def opt_vrednost_za_vec_ponovitev(m,n,a,b,max_stevilo_centrov,stevilo_ponovitev):
    seznam = []
    for i in range(0, stevilo_ponovitev):
        #G = mreza(m,n,a,b)
        razdalje = opt_vrednost_k(m,n,a,b,max_stevilo_centrov)
        seznam.append(razdalje)

    for i in range(len(seznam)):
        while len(seznam[i]) < max_stevilo_centrov:
            seznam[i] = [None] + seznam[i]

    povprecja = []
    for j in range(max_stevilo_centrov):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(round((vsota/stevec),2))

    return seznam, povprecja

#opt_vrednost_za_vec_ponovitev(5,5,3,2,1,5)

In [3]:
import time

#čas izvajanja v odvisnosti od k
def cas_izvajanja_k(m,n,a,b,k):
    G = mreza(m,n,a,b)
    seznam_casov = []
    stevilo_komponent = connected_components_number(G)
    for i in range(stevilo_komponent,k+1):
        zacetni = time.time()
        najkrajsa_razdalja(G,i)
        koncni = time.time() - zacetni
        seznam_casov.append((koncni))
    return seznam_casov

cas_izvajanja_k(3,3,1,1,5)

#vrne povprečne čase(od n ponovitev) za različne k
#FIKSNO: velikost mreže, št. izbrisanih povezav in vozlišč
#SPREMINJAMO: k
def cas_izvajanja_za_vec_ponovitev(m,n,a,b,max_stevilo_centrov,stevilo_ponovitev):
    seznam = []
    for i in range(0, stevilo_ponovitev):
        #G = mreza(m,n,a,b)
        casi = cas_izvajanja_k(m,n,a,b,max_stevilo_centrov)
        seznam.append(casi)

    for i in range(len(seznam)):
        while len(seznam[i]) < max_stevilo_centrov:
            seznam[i] = [None] + seznam[i]

    povprecja = []
    for j in range(max_stevilo_centrov):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)

    return seznam, povprecja

#cas_izvajanja_za_vec_ponovitev(4,4,5,2,7,5)

In [4]:
#kako se optimalna vrednost spreminja glede na št. izbrisanih vozlov a
def opt_vrednost_vozli(m,n,max_st_izbrisanih_vozlisc,b,k):
    seznam_vrednosti = []
    for i in range(0,max_st_izbrisanih_vozlisc + 1):
        G = mreza(m,n,i,b)
        stevilo_komponent = connected_components_number(G)
        if stevilo_komponent <= k:
            razdalja = round(najkrajsa_razdalja(G, k)) #round za zaokrževanje števil 
            seznam_vrednosti.append((razdalja))
        else:
            seznam_vrednosti.append(None)
            
    return seznam_vrednosti

#opt_vrednost_vozli(5,5,15,2,3)

# !NEKI PAMETNGA BO TREBA NAREST Z None-i, ker primerjamo jabolka pa hruske pri razdrobljenih grafih pri relativno majhnih k!
#fukncija vrne opt. razdalje R glede na število odstranjenih vozlišč
#FIKSNO:velikost mreze, K, st. izbrisanih povezav
#SPREMINJAM: st_izbrisanih vozlišč
def R_v_odvisnosti_od_spreminjanje_stevila_vozlisc(m,n,max_st_izbrisanih_vozlisc,b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        razdalje = opt_vrednost_vozli(m,n,max_st_izbrisanih_vozlisc,b,k)
        seznam.append(razdalje)

    povprecja = []
    for j in range(max_st_izbrisanih_vozlisc + 1):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)
    return seznam, povprecja

#R_v_odvisnosti_od_spreminjanje_stevila_vozlisc(5,5,15,2,3,3)

In [2]:
import time

#funkcija drugacna od cas_izvajanja_k(m,n,a,b,k)
#čas izvajanja v odvisnosti od k
def cas_izvajanja_k2(m,n,max_st_izbrisanih_vozlisc,b,k):
    seznam_casov = []
    for i in range(max_st_izbrisanih_vozlisc + 1):
        G = mreza(m,n,i,b)
        stevilo_komponent = connected_components_number(G)
        if stevilo_komponent <= k:
            zacetni = time.time()
            najkrajsa_razdalja(G,k)
            koncni = time.time() - zacetni
            seznam_casov.append(koncni)
        else:
            seznam_casov.append(None)
    return seznam_casov

#cas_izvajanja_k(3,3,1,1,5)

#funkcija vrne povprečne čase (od n ponovitev) glede na spreminjanje vozlov
#FIKSNO:velikost mreze, K, st. izbrisanih povezav
#SPREMINJAM: st_izbrisanih vozlov
def cas_v_odvisnosti_od_spreminjanja_stevila_vozlisc(m,n,max_st_izbrisanih_vozlisc,b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        casi = cas_izvajanja_k2(m,n,max_st_izbrisanih_vozlisc,b,k)
        seznam.append(casi)

    povprecja = []
    for j in range(max_st_izbrisanih_vozlisc + 1):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)
    return seznam, povprecja

#cas_v_odvisnosti_od_spreminjanja_stevila_vozlisc(5,5,15,2,3,3)

In [6]:
#FIKSNO: k, st. izbrisanih vozlisc in povezav
#SPREMINJAVA: velikost mreze
#velikost mreže narašča od 3x3 do nxn
def R_v_odvisnosti_od_velikosti_kvadratne_mreze(n,a,b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        seznamcek = []
        for j in range(3, n+1):
            G = mreza(j,j,a,b)
            stevilo_komponent = connected_components_number(G)
            if stevilo_komponent <= k:
                razdalje = round(najkrajsa_razdalja(G, k))
                seznamcek.append(razdalje)
            else:
                seznamcek.append(None)
        seznam.append(seznamcek)

    povprecja = []
    for j in range(0, n-2):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)

    return seznam, povprecja


#R_v_odvisnosti_od_velikosti_kvadratne_mreze(6,3,2,3,3)

In [7]:
#FIKSNO: k, st. izbrisanih vozlisc in povezav, ŠT. VRSTIC (m)
#SPREMINJAVA: velikost mreze (število stolpcev: od 3 do n)
#velikost mreže narašča od mx3 do mxn
def R_v_odvisnosti_od_velikosti_mreze(m,n,a,b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        seznamcek = []
        for j in range(3, n+1):
            G = mreza(m,j,a,b)
            stevilo_komponent = connected_components_number(G)
            if stevilo_komponent <= k:
                razdalje = round(najkrajsa_razdalja(G, k))
                seznamcek.append(razdalje)
            else:
                seznamcek.append(None)
        seznam.append(seznamcek)

    povprecja = []
    for j in range(0, n-2):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)

    return seznam, povprecja

#R_v_odvisnosti_od_velikosti_mreze(4,6,2,3,3,3)

In [8]:
#FIKSNO: st. izbrisanih vozlisc in povezav
#SPREMINJAVA: velikost mreze in maximalno stevilo K-jev
#velikost mreže narašča od 3x3 do nxn
#VRNE matriko: v a_ij pove povprecni R (za n ponovitev) pri mreži ixi, ki ima j centrov
def R_v_odvisnosti_od_velikosti_kvadratne_mreze_in_k(n,a,b,max_k,stevilo_ponovitev):
    seznam = [] # i-ti element tega seznama pove povprecne R-je za razlicne k-je za ixi matriko
    for i in range(6, n+1):
        seznam1 = [] #seznam povprecnega R za stevilo_centrov = j
        for j in range(1, max_k + 1):
            seznam2 = []
            for p in range(stevilo_ponovitev):
                G = mreza(i,i,a,b)
                stevilo_komponent = connected_components_number(G)
                if stevilo_komponent <= j: #st. komponent <= k (st.centrov) (v tem primeru j)
                    razdalja = round(najkrajsa_razdalja(G, j))
                    seznam2.append(razdalja)
                else:
                    seznam2.append(None)

            vsota = 0
            stevec = 0
            povprecje = 0
            for v in range(len(seznam2)):
                if seznam2[v] != None:
                    vsota += seznam2[v]
                    stevec += 1
            if stevec == 0:
                povprecje = None
            else:
                povprecje = vsota/stevec

            seznam1.append(povprecje)

        seznam.append(seznam1)

    return seznam


#R_v_odvisnosti_od_velikosti_kvadratne_mreze_in_k(6,0,0,9,1) 
#R_v_odvisnosti_od_velikosti_kvadratne_mreze_in_k(15,3,2,3,1) #skenslu po 1400 sekundah gledu sm samo matriko 15x12

In [9]:
import time

#FIKSNO: st. izbrisanih vozlisc in povezav
#SPREMINJAVA: velikost mreze in maximalno stevilo K-jev
#velikost mreže narašča od 3x3 do nxn
#VRNE matriko: v a_ij pove povprecni CAS (za n ponovitev) pri mreži ixi, ki ima j centrov
def cas_v_odvisnosti_od_velikosti_kvadratne_mreze_in_k(n,a,b,max_k,stevilo_ponovitev):
    seznam = [] # i-ti element tega seznama pove povprecne R-je za razlicne k-je za ixi matriko
    for i in range(3, n+1):
        seznam1 = [] #seznam povprecnega R za stevilo_centrov = j
        for j in range(1, max_k + 1):
            seznam2 = []
            for p in range(stevilo_ponovitev):
                G = mreza(i,i,a,b)
                stevilo_komponent = connected_components_number(G)
                if stevilo_komponent <= j: #st. komponent <= k (st.centrov) (v tem primeru j)
                    zacetni = time.time()
                    najkrajsa_razdalja(G, j)
                    koncni = time.time() - zacetni
                    seznam2.append(koncni)
                else:
                    seznam2.append(None)

            vsota = 0
            stevec = 0
            povprecje = 0
            for v in range(len(seznam2)):
                if seznam2[v] != None:
                    vsota += seznam2[v]
                    stevec += 1
            if stevec == 0:
                povprecje = None
            else:
                povprecje = vsota/stevec

            seznam1.append(povprecje)

        seznam.append(seznam1)

    return seznam



#cas_v_odvisnosti_od_velikosti_kvadratne_mreze_in_k(5,3,3,3,3)

In [10]:
%%time

A = mreza(5,5,0,0)
najkrajsa_razdalja(A, 2)

#ZA OBČUTEK:

#A = mreza(8,8,0,0)
#najkrajsa_razdalja(A, 2)
#ČAS: 14min 12s

#A = mreza(7,8,0,0)
#najkrajsa_razdalja(A, 2)
#ČAS: 5min 55s

#A = mreza(7,7,0,0)
#najkrajsa_razdalja(A, 2)
#ČAS: 2min 35s      ČASI NISO VEDNO ISTI, SO PA PODOBNI

#A = mreza(6,6,0,0)
#najkrajsa_razdalja(A, 2)
#ČAS: 24.1 s

#A = mreza(7,7,0,0)
#najkrajsa_razdalja(A, 3)
#ČAS: 4min 28S

#A = mreza(7,7,0,0)
#najkrajsa_razdalja(A, 4)
#ČAS: 5min 8s

#A = mreza(7,7,0,0)
#najkrajsa_razdalja(A, 10)
#ČAS: 7min 43s

#A = mreza(7,7,20,0)
#najkrajsa_razdalja(A, 10)
#ČAS: 1.68 s

#A = mreza(6,6,0,0)
#najkrajsa_razdalja(A, 10)
#ČAS: 1min 12s

#A = mreza(5,5,0,0)
#najkrajsa_razdalja(A, 10)
#ČAS: 8.6 s

#cas_izvajanja_za_vec_ponovitev(5,5,0,0,7,2)
#0.3044382333755493,3.9640878438949585,5.491700053215027,6.347324013710022,7.058931469917297,7.724177837371826,8.292396187782288]

#cas_izvajanja_za_vec_ponovitev(5,5,5,0,7,2)
#[0.12116122245788574,0.8681886196136475,0.6877187490463257,0.9534119367599487,1.16581130027771,1.3667179346084595,1.4272890090942383]

#cas_izvajanja_za_vec_ponovitev(7,7,0,0,2,2)
#([[5.456587791442871, 154.49036502838135],
#  [5.305730819702148, 157.40487551689148]],
# [5.38115930557251, 155.9476202726364])

#cas_izvajanja_za_vec_ponovitev(5,5,5,0,7,2)

CPU times: user 4.32 s, sys: 5.48 ms, total: 4.32 s
Wall time: 8.57 s


In [12]:
#FIKSNO: st. izbrisanih vozlisc in povezav
#SPREMINJAVA: velikost mreze in maximalno stevilo K-jev
#velikost mreže narašča od mx2 do mxn
#VRNE matriko: v a_ij pove povprecni R (za n ponovitev) pri mreži ixi, ki ima j centrov
def R_v_odvisnosti_od_velikosti_mreze_in_k(m,n,a,b,max_k,stevilo_ponovitev):
    seznam = [] # i-ti element tega seznama pove povprecne R-je za razlicne k-je za ixi matriko
    for i in range(2, n+1):
        seznam1 = [] #seznam povprecnega R za stevilo_centrov = j
        for j in range(1, max_k + 1):
            seznam2 = []
            for p in range(stevilo_ponovitev):
                G = mreza(m,i,a,b)
                stevilo_komponent = connected_components_number(G)
                if stevilo_komponent <= j: #st. komponent <= k (st.centrov) (v tem primeru j)
                    razdalja = round(najkrajsa_razdalja(G, j))
                    seznam2.append(razdalja)
                else:
                    seznam2.append(None)

            vsota = 0
            stevec = 0
            povprecje = 0
            for v in range(len(seznam2)):
                if seznam2[v] != None:
                    vsota += seznam2[v]
                    stevec += 1
            if stevec == 0:
                povprecje = None
            else:
                povprecje = vsota/stevec

            seznam1.append(povprecje)

        seznam.append(seznam1)

    return seznam

In [4]:
import time

#funkcija drugacna od cas_izvajanja_k(m,n,a,b,k)
#čas izvajanja v odvisnosti od k
def cas_izvajanja_k3(m,n,a,max_b,k):
    seznam_casov = []
    for i in range(max_b + 1):
        G = mreza(m,n,a,i)
        stevilo_komponent = connected_components_number(G)
        if stevilo_komponent <= k:
            zacetni = time.time()
            najkrajsa_razdalja(G,k)
            koncni = time.time() - zacetni
            seznam_casov.append(koncni)
        else:
            seznam_casov.append(None)
    return seznam_casov

#cas_izvajanja_k(3,3,1,1,5)

#funkcija vrne povprečne čase (od n ponovitev) glede na spreminjanje vozlov
#FIKSNO:velikost mreze, K, st. izbrisanih povezav
#SPREMINJAM: st_izbrisanih vozlov
def cas_v_odvisnosti_od_spreminjanja_stevila_povezav(m,n,a,max_b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        casi = cas_izvajanja_k3(m,n,a,max_b,k)
        seznam.append(casi)

    povprecja = []
    for j in range(max_b + 1):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)
    return seznam, povprecja

#cas_v_odvisnosti_od_spreminjanja_stevila_povezav(4,4,0,10,3,3)